In [ ]:
import pandas as pd
import numpy as np
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization
from keras.models import Model
import matplotlib.pyplot as plt
import gc
from sklearn.model_selection import train_test_split



gc.collect()
gc.set_threshold(10000)






In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Reshape, UpSampling2D, BatchNormalization
from keras.models import Model
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
import gc

# Загрузка данных
tma_image_paths = pd.read_csv('/root/ubc_ocean/alper/models/paths/tma_image_paths.csv')
train_labels = pd.read_csv('/root/ubc/train.csv')

# Определение пути к изображениям
tma_image_paths['image_id'] = tma_image_paths['img_path'].apply(lambda x: os.path.splitext(os.path.basename(x))[0])
train_labels['image_id'] = train_labels['image_id'].astype(str)
merged_data = pd.merge(tma_image_paths, train_labels, on='image_id')

# Разделение данных
grouped = merged_data.groupby('label')
test_data = pd.concat([group.sample(1, random_state=42) for name, group in grouped])
train_val_data = merged_data.drop(test_data.index)
x_train_df, x_val_df = train_test_split(train_val_data, test_size=0.2, random_state=42)

# Предобработка изображений
def preprocess_image(image_path, target_size=(224, 224)):
    img = load_img(image_path, target_size=target_size)
    img = img_to_array(img)
    img = img / 255.0
    return img

x_train = np.array([preprocess_image(img_path) for img_path in x_train_df['img_path']])
x_val = np.array([preprocess_image(img_path) for img_path in x_val_df['img_path']])
x_test = np.array([preprocess_image(img_path) for img_path in test_data['img_path']])

# Параметры энкодера
encoding_dim = 128

# Архитектура энкодера
encoder_input = Input(shape=(224, 224, 3))
x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoder_input)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Flatten()(x)
encoded = Dense(encoding_dim, activation='relu')(x)
encoder = Model(encoder_input, encoded)

# Архитектура декодера
decoder_input = Input(shape=(encoding_dim,))
x = Dense(7 * 7 * 128)(decoder_input) # Подберите соответствующее число
x = Reshape((7, 7, 128))(x)
x = UpSampling2D((2, 2))(x) # Увеличиваем до 14x14
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x) # Увеличиваем до 28x28
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x) # Увеличиваем до 56x56
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x) # Увеличиваем до 112x112
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x) # Увеличиваем до 224x224
decoded_output = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)
decoder = Model(decoder_input, decoded_output)





# Создание автокодировщика
autoencoder_input = Input(shape=(224, 224, 3))
encoded_img = encoder(autoencoder_input)
decoded_img = decoder(encoded_img)
autoencoder = Model(autoencoder_input, decoded_img)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Обучение автокодировщика
autoencoder.fit(x_train, x_train, epochs=50, batch_size=128, shuffle=True, validation_data=(x_val, x_val))

# Кодирование изображений
encoded_imgs = encoder.predict(x_train)

# Добавление шума
epsilon = 0.03
noisy_encoded_imgs = encoded_imgs + epsilon * np.random.normal(loc=0.0, scale=1.0, size=encoded_imgs.shape)

# Декодирование изображений
generated_imgs = decoder.predict(noisy_encoded_imgs)

# Визуализация изображений
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(generated_imgs[i])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

# Сохранение изображений
def save_generated_images(images, save_path, prefix='gen', format='jpeg'):
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    for i, img in enumerate(images):
        img_path = os.path.join(save_path, f"{prefix}_{i}.{format}")
        plt.imsave(img_path, img)

#save_path = '/root/ubc_ocean/Vlad/Created_Pictures'
#save_generated_images(generated_imgs, save_path)

# Очистка памяти
gc.collect()
gc.set_threshold(10000)
